## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [31]:
NAME = "Αλεξανδρίδης Φώτιος"
AEM = "9953"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [32]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

#Load Breast Cancer dataset from scikit-learn
data = load_breast_cancer()

#Split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.3, random_state=0)

In [33]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [34]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        # Keep the base estimator
        self.estimator = estimator

    def fit(self, X_train, y_train):
        # Store the number of features in the dataset 
        self.n_features_ = X_train.shape[1]
        self.estimators_ = []

        # Create a separate model for each feature
        for i in range(self.n_features_):  
            # Create a copy of the base estimator
            estimator_i = clone(self.estimator)
            
            # Ignore one feature at a time for training
            mask = np.ones(self.n_features_, dtype=bool)
            mask[i] = False  
            X_train_new = X_train[:, mask]

            estimator_i.fit(X_train_new, y_train)
            self.estimators_.append(estimator_i)

    def predict(self, X):
        predictions = []
        for i, estimator in enumerate(self.estimators_):
            # Ignore one feature at a time for prediction
            mask = np.ones(self.n_features_, dtype=bool)
            mask[i] = False  
            X_new = X[:, mask]
            predictions.append(estimator.predict(X_new))
      
        # Combine predictions from all models using majority vote
        predictions = np.array(predictions)
        return np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), 0, predictions)   

In [35]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [36]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

# Do this since adaboost expects the classes to be [-1, 1] and not [0, 1]
# Will need this for below implementation as well
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        # Hold arguments
        self.estimator = estimator
        self.n_estimators = n_estimators
        # Used to hold values needed
        self.estimators = []
        self.alphas = []

    def fit(self, X_train, y_train):
        # Get number of samples and initialize weights
        n_samples = X_train.shape[0]
        weights = np.full(n_samples, (1 / n_samples))
        for _ in range(self.n_estimators):
            # Clone a new base estimator
            estimator = clone(self.estimator)
            # Use previous weights
            estimator.fit(X_train, y_train, sample_weight=weights)
            self.estimators.append(estimator)
            predictions = estimator.predict(X_train)

            # Calculate prediction errors
            errors = np.where(predictions == y_train, 0, 1)

            # Calculate weighted error
            weighted_error = np.dot(weights, errors) / weights.sum()

            # Break condition
            if weighted_error >= 0.5 or weighted_error == 0:
                continue

            # Calculate alpha
            alpha = np.log((1.0 - weighted_error) / weighted_error)
            self.alphas.append(alpha)

            # Update weights
            weights = weights * np.exp(alpha * errors)

            # Normalize to one
            weights = weights / np.sum(weights)

    def predict(self, X):
        # Get the number of samples
        n_samples = X.shape[0]
        # Initialize predictions output
        predictions = np.zeros(n_samples)
        # Calculate predictions
        for estimator, alpha in zip(self.estimators, self.alphas):
            predictions += alpha * estimator.predict(X)
        # Return predictions
        return np.where(predictions >= 0, 1, -1)

In [37]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591

In [38]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591